***
# Logistic Regression and Support Vector Machines
MSDS 7331-407, Lab 2  
*Jenna Ford, Edward Fry, Christian Nava, and Jonathan Tan* 
***

## Table of Contents

<a href='#Section_1'> 1. Preparation and Dataset Loading </a>  
<a href='#Section_2'> 2. Data Cleansing (Lab 1) </a>  
<a href='#Section_3'> 3. Create A Logistic Regression Model and a Support Vector Machine Model </a>  
<a href='#Section_4'> 4. Model Advantages </a>  
<a href='#Section_5'> 5. Weight Interpretation from Logistic Regression Model</a>  
<a href='#Section_6'> 6. Insights from Support Vectors </a>  

<!-- <a href='#Section_4_a'> &nbsp;&nbsp;&nbsp; a. Missing Values </a>  --> 
<!-- <a href='#Section_4_c_i'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; i. Cross Street </a>    --> 

<a id = 'Section_1'></a>

## 1. Preparation and Dataset Loading

In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Creating training and test sets
import sklearn

# File system management
import os.path

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Constants
DATAPATH_BASE = 'https://machinelearningi.blob.core.windows.net/group-project/'
DATAPATH_SAS_TOKEN = '?sv=2019-02-02&ss=bfqt&srt=sco&sp=rwdlacup&se=2020-04-27T11:12:37Z&st=2020-01-23T04:12:37Z&spr=https&sig=jpIpjrp8dIg9eyUyPpmgTe5yj9i1ZoCSru5kBVHcUO8%3D'
DATAPATH_FILENAME = 'Arrest_Data_from_2010_to_Present.csv'

# Fully qualified paths ready to use
DATA_SOURCE = "".join([DATAPATH_BASE, DATAPATH_FILENAME, DATAPATH_SAS_TOKEN])

# Options
pd.set_option('float_format', '{:.2f}'.format)  # Reign in the scientific notation for reasonable values

# Load data for analysis; only read if needed because the import can take a long time
try:
    if len(df.index) < 1:
        df_raw = pd.read_csv(DATA_SOURCE) # If we get here, the dataframe was empty
except:   
    df_raw = pd.read_csv(DATA_SOURCE) # If we get here, the dataframe did not exist
    

In [3]:
df = df_raw
print("The dataset has {:,} rows and {:,} columns".format(*df.shape))

The dataset has 1,326,626 rows and 17 columns


<a id = 'Section_2'></a>

## 2. Data Cleansing (Lab 1)

The data cleansing steps performed below are repeated from Lab 1.

In [4]:
# Time - filter out 0 and missing
df = df[df['Time'] != 0]
df['Time'] = df['Time'].astype(str) 
df = df[df['Time'] != 'nan']

# Time - Convert float to string. Get rid of decimals. Replace missing or invalid values with '0000'.
df['Time'] = df['Time'].astype(str).str.split(".", expand = True)[0].replace(to_replace = ['2400','nan'], value = '0000') 

# Time - Fill time column with leading zeros to have 4 characters total
df['Time'] = df['Time'].apply(lambda x: '{0:0>4}'.format(x))

# Time - Add colon to Time values by converting attribute to a datetime variable 
df['Time'] = pd.to_datetime(df['Time'], format = '%H%M').dt.time

# Age - Drop the observations where Age is less than 16
df.drop(df[df['Age'] < 16].index, inplace = True) 

# Arrest Type Code - Drop the observations where Arrest Type Code = 'D'
df.drop(df[df['Arrest Type Code'] == 'D'].index, inplace = True) 

# Descent Code - Re-classify any descent not in (B,H,O,W) into 0
descent_list = ['B','H','O','W']
df['Descent Code'] = np.where(np.isin(df['Descent Code'],descent_list),df['Descent Code'],'O')

# Get hour
df['Hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour

# Convert Arrest Date to datetime
df['Arrest Date'] = pd.to_datetime(df['Arrest Date'])

# Extract year, month, and day of week and add to dataframe as new attributes
df['arrest_year']= df['Arrest Date'].dt.year
df['arrest_month']= df['Arrest Date'].dt.month
df['arrest_day_of_week'] = df['Arrest Date'].dt.weekday_name

# remove unecessary columns
df.drop(['Cross Street','Charge Description','Charge','Charge Group Description','Time',
         'Arrest Date','Report ID','Address','Area Name'], axis=1, inplace=True)

# Change data types
df['Age'] = df['Age'].astype(np.int8)
df['Reporting District'] = df['Reporting District'].astype(np.str)
df['Area ID'] = df['Area ID'].astype(np.str)
df['Charge Group Code'] = df['Charge Group Code'].astype(np.str)
df['Hour'] = df['Hour'].astype(np.str)
df['arrest_year'] = df['arrest_year'].astype(np.str)
df['arrest_month'] = df['arrest_month'].astype(np.str)
df['arrest_day_of_week'] = df['arrest_day_of_week'].astype(np.str)

# print clean dataset
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1278805 entries, 0 to 1326625
Data columns (total 12 columns):
Area ID               1278805 non-null object
Reporting District    1278805 non-null object
Age                   1278805 non-null int8
Sex Code              1278805 non-null object
Descent Code          1278805 non-null object
Charge Group Code     1278805 non-null object
Arrest Type Code      1278805 non-null object
Location              1278805 non-null object
Hour                  1278805 non-null object
arrest_year           1278805 non-null object
arrest_month          1278805 non-null object
arrest_day_of_week    1278805 non-null object
dtypes: int8(1), object(11)
memory usage: 118.3+ MB


In [5]:
print("The clean data set has {:,} rows and {:,} columns".format(*df.shape))

The clean data set has 1,278,805 rows and 12 columns


<a id = 'Section_3'></a>

## 3. Create A Logistic Regression Model and a Support Vector Machine Model

In order to perform a logistic regression, all categorical variables need to be dummy encoded. For example, if a variable has 3 different categories we need to split this out to 3 different variables. The code below does this for the categorical variables used in the model. The target variable `Arrest Type Code` is transformed to range from 0-3 instead of alphabetical letters representing the arrest type. Also, `Age` was bucketed into 10 year increments.

In [6]:
# Encode Arrest Type Code as Categorical
cleanup_arrest = {"Arrest Type Code": {"F": 0, "M": 1, "I": 2, "O":3}}
df.replace(cleanup_arrest,inplace=True)

# Create buckets for Age
df['age_range'] = pd.cut(df.Age,[16,25,35,45,55,65,75,1e6],4,labels=[0,1,2,3,4,5,6]) # this creates a new variable
df['age_range'] = df.age_range.astype(np.int)

# Perform one-hot encoding of the categorical data "Descent Code"
tmp_df = pd.get_dummies(df['Descent Code'],prefix='Descent')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# Replace the current Sex atribute with something slightly more intuitive and readable
df['IsMale'] = df['Sex Code']=='M' 
df.IsMale = df.IsMale.astype(np.int)

# Perform one-hot encoding of the categorical data "DOW"
tmp_df = pd.get_dummies(df['arrest_day_of_week'],prefix='DOW')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# Perform one-hot encoding of the categorical data "DOW"
tmp_df = pd.get_dummies(df['Area ID'],prefix='Area')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# Perform one-hot encoding of the categorical data "DOW"
tmp_df = pd.get_dummies(df['Charge Group Code'],prefix='Charge')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

After properly encoding all variables, we remove the original variables from the dataset. We also remove any variables that will not be used in this model. Information about the resulting dataframe is printed for verification and the first 5 records are printed out.

Note that this is just a first pass at the model, other variables will be used in future iterations.

In [8]:
df_logreg = df

df.drop(['Sex Code','Descent Code','arrest_day_of_week','Area ID','Reporting District','Charge Group Code',
         'Location','arrest_year','Age','Hour','arrest_month'], axis=1, inplace=True)
    
df_logreg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1278805 entries, 0 to 1326625
Data columns (total 64 columns):
Arrest Type Code    1278805 non-null int64
age_range           1278805 non-null int32
Descent_B           1278805 non-null uint8
Descent_H           1278805 non-null uint8
Descent_O           1278805 non-null uint8
Descent_W           1278805 non-null uint8
IsMale              1278805 non-null int32
DOW_Friday          1278805 non-null uint8
DOW_Monday          1278805 non-null uint8
DOW_Saturday        1278805 non-null uint8
DOW_Sunday          1278805 non-null uint8
DOW_Thursday        1278805 non-null uint8
DOW_Tuesday         1278805 non-null uint8
DOW_Wednesday       1278805 non-null uint8
Area_1              1278805 non-null uint8
Area_10             1278805 non-null uint8
Area_11             1278805 non-null uint8
Area_12             1278805 non-null uint8
Area_13             1278805 non-null uint8
Area_14             1278805 non-null uint8
Area_15             1278805

In [9]:
df.head()

,Arrest Type Code,age_range,Descent_B,Descent_H,Descent_O,Descent_W,IsMale,DOW_Friday,DOW_Monday,DOW_Saturday,...,Charge_29.0,Charge_3.0,Charge_4.0,Charge_5.0,Charge_6.0,Charge_7.0,Charge_8.0,Charge_9.0,Charge_99.0,Charge_nan
0,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,2,0,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,0,1,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,0,2,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


The section of code below sets up the frame for the logistic regression model. The target `Arrest Type Code` is identified and all other variables are assigned as explanatory variables.

Also indicated in this section of code is the training/test split of 80%/20%. The model will be built with 80% of the data and tested on the remaining 20%.

Note that only one iteration of the model is being run. This is due to the time is takes to run the model with the number of observations and variables we have in the dataset. In future assignments, we will run through more iterations.

In [ ]:
#training/test split
from sklearn.model_selection import ShuffleSplit

# we want to predict the X and y data as follows:
if 'Arrest Type Code' in df_logreg:
    y = df_logreg['Arrest Type Code'].values # get the labels we want
    del df_logreg['Arrest Type Code'] # get rid of the class label
    X = df_logreg.values # use everything else to predict!

    ## X and y are now numpy matrices, by calling 'values' on the pandas data frames we
    #    have converted them into simple matrices to use with scikit learn
    
    
# to use the cross validation object in scikit learn, we need to grab an instance
#    of the object and set it up. This object will be able to split our data into 
#    training and testing splits
num_cv_iterations = 1
num_instances = len(y)
cv_object = ShuffleSplit(n_splits=num_cv_iterations,
                         test_size  = 0.2)
                         
print(cv_object)

The code below splits the data into the training/test split identified earlier and runs a multi-nomial logistic regression.

In [ ]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(multi_class='multinomial', solver='newton-cg' ) # get object

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

iter_num=0
# the indices are the rows used for training and testing in each iteration
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    lr_clf.fit(X[train_indices],y[train_indices])  # train object
    y_hat = lr_clf.predict(X[test_indices]) # get test set precitions

    # print the accuracy and confusion matrix 
    print("====Iteration",iter_num," ====")
    print("accuracy", mt.accuracy_score(y[test_indices],y_hat)) 
    print("confusion matrix\n",mt.confusion_matrix(y[test_indices],y_hat))

The accuracy for this iteration of the model is 75.9%. Without adjusting model parameters, we are already very close to our target accuracy of 75%. The various iterations we have run ranged from accuracy of 70% to 78%.

It is interesting to note that the model never predicts the arrest type codes I-Infraction and O-Other. In future iterations, we will try to improve this, but it may be difficult due to the limited number of observations available for the arrest types.